## Next Word Prediction

In [128]:
import re
import math
from collections import defaultdict, Counter


In [169]:
telugu_sentences = [
    "రాజు పుస్తకం చదువుతున్నాడు",
    "సూర్యుడు ఉదయమవుతోంది",
    "పిల్లలు పార్క్ లో ఆడుతున్నారు",
    "రాము మరియు నందిని సముద్ర తీరంలో తిరుగుతున్నారు",
    "ఆమె వంటగదిలో కుక్కీ తయారు చేస్తోంది",
    "సైనికులు తమ విధులు నిబద్ధతగా చేస్తున్నారు",
    "బెంగళూరు లో కొత్త విద్యుత్ ప్రాజెక్ట్ ప్రారంభమైంది",
    "అతను నేటి వార్తల గురించి చదువుతున్నాడు",
    "వర్షం కారణంగా ఆట రద్దు అయ్యింది",
    "రాజు మరియు రాము బ్లాక్ లో ఆడుతున్నారు"
]


In [170]:
def tokenize_telugu_sentence(sentence):
    return [w for w in sentence.split() if re.match(r'[\u0C00-\u0C7F]+', w)]

In [171]:
tokenize_telugu_sentence(telugu_sentences[0])

['రాజు', 'పుస్తకం', 'చదువుతున్నాడు']

In [172]:
tokenized_sentences = [tokenize_telugu_sentence(s) for s in telugu_sentences]

In [133]:
for i, t in enumerate(tokenized_sentences[:2]):
    print(f"Sentence {i+1} tokens:", t)

Sentence 1 tokens: ['రాజు', 'పుస్తకం', 'చదువుతున్నాడు']
Sentence 2 tokens: ['సూర్యుడు', 'ఉదయమవుతోంది']


In [134]:
bigram_counts = defaultdict(Counter)

for tokens in tokenized_sentences:
    tokens = ["<s>"] + tokens + ["</s>"]
    for i in range(len(tokens)-1):
        bigram_counts[tokens[i]][tokens[i+1]] += 1

In [135]:
bigram_counts["<s>"].most_common(3)

[('రాజు', 2), ('సూర్యుడు', 1), ('పిల్లలు', 1)]

In [136]:
trigram_counts = defaultdict(Counter)
for tokens in tokenized_sentences:
    tokens = ["<s>", "<s>"] + tokens + ["</s>"]
    for i in range(len(tokens)-2):
        context = (tokens[i], tokens[i+1])
        trigram_counts[context][tokens[i+2]] += 1

In [137]:
trigram_counts[('<s>','<s>')]

Counter({'రాజు': 2,
         'సూర్యుడు': 1,
         'పిల్లలు': 1,
         'రాము': 1,
         'ఆమె': 1,
         'సైనికులు': 1,
         'బెంగళూరు': 1,
         'అతను': 1,
         'వర్షం': 1})

In [138]:
context = ("రాము", "బ్లాక్")
print("Next words for context", context, ":", trigram_counts[context].most_common(3))

Next words for context ('రాము', 'బ్లాక్') : [('లో', 1)]


In [139]:
def predict_next_word(context, trigram_counts, top_k=3):
    candidates = trigram_counts.get(context, {})
    total = sum(candidates.values())
    if not total:
        return []
    probs = {w: c/total for w,c in candidates.items()}
    return sorted(probs.items(), key=lambda x: x[1], reverse=True)[:top_k]

In [140]:
predict_next_word(("రాము", "బ్లాక్"), trigram_counts)

[('లో', 1.0)]

In [141]:
def compute_perplexity(tokenized_sentences, trigram_counts, vocab_size):
    N = 0
    log_prob_sum = 0
    for tokens in tokenized_sentences:
        tokens = ["<s>", "<s>"] + tokens + ["</s>"]
        for i in range(2, len(tokens)):
            context = (tokens[i-2], tokens[i-1])
            word = tokens[i]
            count_context = sum(trigram_counts.get(context, {}).values())
            word_count = trigram_counts.get(context, {}).get(word, 0)
            prob = (word_count + 1)/(count_context + vocab_size)
            log_prob_sum += -math.log(prob)
            N += 1
    return math.exp(log_prob_sum/N)

In [142]:
vocab = set(w for s in tokenized_sentences for w in s) | {"<s>", "</s>"}
compute_perplexity(tokenized_sentences, trigram_counts, len(vocab))


21.640217955836274

In [166]:
class NGramModel:
    def __init__(self, n, lambda_trigram=0.6, lambda_bigram=0.3, lambda_unigram=0.1):
        self.n = n
        self.ngrams = defaultdict(Counter)
        self.vocab = set()
        self.unigram_counts = Counter()
        self.lambda1 = lambda_trigram
        self.lambda2 = lambda_bigram
        self.lambda3 = lambda_unigram

        total = self.lambda1 + self.lambda2 + self.lambda3
        if abs(total - 1) > 0.001:
            self.lambda1 = 0.6
            self.lambda2 = 0.3
            self.lambda3 = 0.1
    def train(self, tokenized_sentences):
        self.unigram_counts = Counter()
        self.vocab = set()

        if tokenized_sentences and isinstance(tokenized_sentences[0], str):
            tokenized_sentences = [self.tokenize_telugu_sentence(s) for s in tokenized_sentences]


        for tokens in tokenized_sentences:
            for word in tokens:
                self.unigram_counts[word] += 1
                self.vocab.add(word)


        for tokens in tokenized_sentences:
            tokens = ["<s>"] * (self.n - 1) + tokens + ["</s>"]
            self.vocab.update(tokens)
            for i in range(len(tokens) - self.n + 1):
                context = tuple(tokens[i:i+self.n-1])
                target = tokens[i+self.n-1]
                self.ngrams[context][target] += 1

    def tokenize_telugu_sentence(self, sentence):
        return [w for w in sentence.split() if re.match(r'[\u0C00-\u0C7F]+', w)]

    def predict_next(self, context, top_k=3):
        all_predictions = {}
        unigram_total = sum(self.unigram_counts.values())
        
        candidate_words = set()
        if len(context) >= self.n - 1:
            ngram_context = tuple(context[-(self.n-1):])
            ngram_data = self.ngrams.get(ngram_context, {})
            ngram_total = sum(ngram_data.values())
            
            if ngram_total > 0:
                for word, count in ngram_data.items():
                    if word not in ['<s>', '</s>']:
                        ngram_prob = count / ngram_total
                        all_predictions[word] = self.lambda1 * ngram_prob
                        candidate_words.add(word)

        for word, count in self.unigram_counts.most_common(100):
            if word not in ['<s>', '</s>'] and word not in candidate_words:
                unigram_prob = count / unigram_total
                all_predictions[word] = self.lambda3 * unigram_prob
        
        if all_predictions:
            total_prob = sum(all_predictions.values())
            if total_prob > 0:
                normalized_predictions = {word: prob/total_prob for word, prob in all_predictions.items()}
                sorted_predictions = sorted(normalized_predictions.items(), key=lambda x: x[1], reverse=True)
                return sorted_predictions[:top_k]
        
        return []


    def perplexity(self, test_sentences):
        if test_sentences and isinstance(test_sentences[0], str):
            test_sentences = [self.tokenize_telugu_sentence(s) for s in test_sentences]

        N = 0
        log_prob_sum = 0
        unigram_total = sum(self.unigram_counts.values())

        for tokens in test_sentences:
            tokens = ["<s>"] * (self.n - 1) + tokens + ["</s>"]
            for i in range(self.n - 1, len(tokens)):
                context = tuple(tokens[i - self.n + 1:i])
                word = tokens[i]
                ngram_prob = 0
                ngram_data = self.ngrams.get(context, {})
                ngram_total = sum(ngram_data.values())
                if ngram_total > 0:
                    ngram_prob = ngram_data.get(word, 0) / ngram_total
                unigram_prob = self.unigram_counts.get(word, 0) / unigram_total
                prob = max(self.lambda1 * ngram_prob + self.lambda3 * unigram_prob, 1e-10)
                log_prob_sum += -math.log(prob)
                N += 1

        return math.exp(log_prob_sum / N) if N > 0 else float('inf')

In [173]:
model = NGramModel(n=3)
model.train(tokenized_sentences)

In [174]:
print(len(tokenized_sentences))
print(sum(len(s) for s in tokenized_sentences))


10
47


In [176]:

phrases = [
    ["రాజు"],
    ["పిల్లలు", "పార్క్"],
    ["రాము", "మరియు"],
    ["ఆమె", "వంటగదిలో"],
    ["బెంగళూరు", "లో"]
]

In [177]:
for phrase in phrases:
    preds = model.predict_next(phrase)
    print("Input:", " ".join(phrase))
    if preds:
        for word, p in preds:
            print(f"{word}:({p:.2f})")
    else:
        print("No prediction found")
    print()

model.perplexity(tokenized_sentences)

Input: రాజు
లో:(0.06)
రాజు:(0.04)
చదువుతున్నాడు:(0.04)

Input: పిల్లలు పార్క్
లో:(0.87)
రాజు:(0.01)
చదువుతున్నాడు:(0.01)

Input: రాము మరియు
నందిని:(0.86)
లో:(0.01)
రాజు:(0.01)

Input: ఆమె వంటగదిలో
కుక్కీ:(0.86)
లో:(0.01)
రాజు:(0.01)

Input: బెంగళూరు లో
కొత్త:(0.86)
లో:(0.01)
రాజు:(0.01)



2.4714303982860715

## Sentence Completion

In [178]:
def complete_sentence(model, context_list, max_len=20):
    completed = context_list.copy()
    for _ in range(max_len):
        if len(completed) < model.n - 1:
            current_context = ["<s>"] * (model.n - 1 - len(completed)) + completed
        else:
            current_context = completed[-(model.n-1):]

        context_tuple = tuple(current_context)
        candidates = model.ngrams.get(context_tuple, {})
        if not candidates:
            if len(completed) > 0:
                context_bi = tuple(completed[-1:])
                candidates = model.ngrams.get(context_bi, {})
        if not candidates:
            break
        total = sum(candidates.values())
        probs = {w: c/total for w, c in candidates.items()}
        next_word = max(probs, key=probs.get)
        if next_word == "</s>":
            break
        completed.append(next_word)
    return completed

In [179]:
context= ["రాజు"]
model=NGramModel(3)
model.train(tokenized_sentences)
completed_sentence = complete_sentence(model, context)
print(" ".join(completed_sentence))

రాజు పుస్తకం చదువుతున్నాడు


## Levishtein Distance

In [180]:
def levenshtein_distance(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m+1): dp[i][0] = i
    for j in range(n+1): dp[0][j] = j
    for i in range(1, m+1):
        for j in range(1, n+1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
    return dp[m][n]

In [181]:
word = "రాజు"
candidates = ["రాజు", "రాజి", "రాజ"]
distances = {w: levenshtein_distance(word, w) for w in candidates}
print(distances)

{'రాజు': 0, 'రాజి': 1, 'రాజ': 1}


In [182]:
class TeluguSpellChecker:
    def __init__(self, vocabulary):
        self.vocabulary = vocabulary

    def levenshtein_distance(self, s1, s2):
        m, n = len(s1), len(s2)
        dp = [[0]*(n+1) for _ in range(m+1)]
        for i in range(m+1):
            dp[i][0] = i
        for j in range(n+1):
            dp[0][j] = j
        for i in range(1, m+1):
            for j in range(1, n+1):
                if s1[i-1] == s2[j-1]:
                    dp[i][j] = dp[i-1][j-1]
                else:
                    dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
        return dp[m][n]

    def suggest_corrections(self, word, max_suggestions=3):
        if word in self.vocabulary:
            return []
        suggestions = []
        for correct_word in self.vocabulary:
            dist = self.levenshtein_distance(word, correct_word)
            if dist <= 2:
                suggestions.append((correct_word,dist))
        suggestions.sort(key=lambda x: x[1])
        return [word for word, dist in suggestions[:max_suggestions]]

    def check_sentence(self, sentence):
        if isinstance(sentence, str):
            words = [w for w in sentence.split() if re.match(r'[\u0C00-\u0C7F]+', w)]
        else:
            words=sentence

        corrections = {}
        for word in sentence.split():
            suggestions = self.suggest_corrections(word)
            if suggestions:
                corrections[word] = suggestions
        return corrections

## Code Checking

In [183]:
model = NGramModel(n=3)
model.train(tokenized_sentences)
phrases = [["రాజు"], ["పిల్లలు", "పార్క్"], ["రాము", "మరియు"]]
for phrase in phrases:
    preds = model.predict_next(phrase)
    print(f"{' '.join(phrase)}: {[p[0] for p in preds]}")

vocab = model.vocab   
completed = complete_sentence(model, ["రాము"])
print(f"Completed Sentence: {' '.join(completed)}")
spell_checker = TeluguSpellChecker(vocab)
test_sentences = [
        "రాజు పుస్తకం చదువుతున్నాడు",
        "రాజి పుస్తక చదువుతున్నాడు",
        "పిల్లలు పార్క్ లో ఆడుతున్నారు"
    ]
for sentence in test_sentences:
    corrections=spell_checker.check_sentence(sentence)
    if corrections:
        print(f"Actual Sentence: {sentence}")
        print(f"Corrected_sentence: {corrections}")
    else:
        print(f"Sentence {sentence} is already correct")

print(f"Perplexity: {model.perplexity(tokenized_sentences):.2f}")



రాజు: ['లో', 'రాజు', 'చదువుతున్నాడు']
పిల్లలు పార్క్: ['లో', 'రాజు', 'చదువుతున్నాడు']
రాము మరియు: ['నందిని', 'లో', 'రాజు']
Completed Sentence: రాము మరియు నందిని సముద్ర తీరంలో తిరుగుతున్నారు
Sentence రాజు పుస్తకం చదువుతున్నాడు is already correct
Actual Sentence: రాజి పుస్తక చదువుతున్నాడు
Corrected_sentence: {'రాజి': ['రాజు', 'రాము'], 'పుస్తక': ['పుస్తకం']}
Sentence పిల్లలు పార్క్ లో ఆడుతున్నారు is already correct
Perplexity: 2.47


In [193]:
import pickle
import json
#rewrite this function if json works..remove other part of function accordingly 
def load_telugu_data():
    sentences = []
    try:
        with open('telugu_tokenized_sentences.json', 'r', encoding='utf-8') as f:       #if json works correctly, function returns 
            tokenized_sentences = json.load(f)
        print(f"Loaded {len(tokenized_sentences)} pre-tokenized sentences from JSON")
        return tokenized_sentences
    except Exception as e:
        print(f"Pre-tokenized JSON not available and exited with error {e}")

    try:
        with open('Data/final_cleaned_telugu_data.txt', 'r', encoding='utf-8') as f:             #directly strip sentences from cleaned data
            sentences = [line.strip() for line in f if line.strip()]
        print(f"Loaded {len(sentences)} sentences from cleaned data")
    except:
        print("Cleaned data file not available")

    if not sentences:                                                                      #if not, load the data from telugu_sentences
        try:
            with open('telugu_sentences.txt', 'r', encoding='utf-8') as f:
                sentences = [line.strip() for line in f if line.strip()]
            print(f"Loaded {len(sentences)} sentences from sentences file")
        except :
            print(f"Sentences file not available, using sample data")
            sentences = [
                "రాజు పుస్తకం చదువుతున్నాడు",
                "సూర్యుడు ఉదయమవుతోంది",
                "పిల్లలు పార్క్ లో ఆడుతున్నారు",
                "రాము మరియు నందిని సముద్ర తీరంలో తిరుగుతున్నారు",
                "ఆమె వంటగదిలో కుక్కీ తయారు చేస్తోంది"
            ]

    if sentences and isinstance(sentences[0], str):
        tokenized_sentences = [tokenize_telugu_sentence(s) for s in sentences]
        print(f"Tokenized {len(tokenized_sentences)} sentences")
        return tokenized_sentences

    return sentences

def load_telugu_vocabulary():
    try:
        with open('telugu_vocabulary.txt', 'r', encoding='utf-8') as f:
            vocabulary = set(line.strip() for line in f if line.strip())
        print(f"Loaded {len(vocabulary)} words from vocabulary file")
        return vocabulary
    except:
        print("Vocabulary file not available, will use model vocabulary")
        return None



tokenized_sentences = load_telugu_data()

Pre-tokenized JSON not available and exited with error Invalid control character at: line 3 column 7 (char 12)
Loaded 179132 sentences from cleaned data
Tokenized 179132 sentences


In [155]:
def save_ngram_model(model, filename='telugu_ngram_model.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model saved to {filename}")

def load_ngram_model(filename='telugu_ngram_model.pkl'):
    try:
        with open(filename, 'rb') as f:
            model = pickle.load(f)
        print(f"Model loaded from {filename}")
        return model
    except:
        print(f"Could not load model from {filename}")
        return None

In [188]:
model = NGramModel(n=3)
model.train(tokenized_sentences)


wiki_phrases = [
    ["భారత", "దేశం"],
    ["తెలుగు", "భాష"], 
    ["హైదరాబాద్", "లో"]
]

for phrase in wiki_phrases:
    preds = model.predict_next(phrase, top_k=3)
    print(f"  {' '.join(phrase)}: {[p[0] for p in preds]}")
    

print(f"\nPERFORMANCE METRICS:")
print(f"  Vocabulary size: {len(model.vocab)} words")
print(f"  Training sentences: {len(tokenized_sentences)}")
print(f"  Perplexity: {model.perplexity(tokenized_sentences):.2f}")

  భారత దేశం: ['మొత్తం', 'నుండి', 'అవుతుంది.']
  తెలుగు భాష: ['మాట్లాడే', 'చరిత్ర', 'ఈ']
  హైదరాబాద్ లో: ['ఉండగా', 'అమీర్', 'జీవిస్తున్నారు.']

PERFORMANCE METRICS:
  Vocabulary size: 276347 words
  Training sentences: 179132
  Perplexity: 6.23


In [ ]:
#save_ngram_model(model)

Model saved to telugu_ngram_model.pkl


In [194]:

model = load_ngram_model()

if model is None:
    model = NGramModel(n=3)
    model.train(tokenized_sentences)
    save_ngram_model(model)


Model loaded from telugu_ngram_model.pkl


In [195]:
print(f"Sentence Completion:")
completed = complete_sentence(model, ["రాజు"])
print(f"Completed Sentence: {' '.join(completed)}")

Sentence Completion:
Completed Sentence: రాజు తన పాలనను దైవిక మూలం నుండి దిగడం ద్వారా కాకుండా, బౌద్ధ సంఘం ఆమోదం పొందడం ద్వారా సంపాదించడానికి ప్రయత్నించాడు.
